In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## Computing Devices

In [2]:
def cpu(): #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0): #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [3]:
def num_gpus(): #@save
    """Get the number of available GPUs."""
    return torch.cuda.device_count()

num_gpus()

1

In [4]:
def try_gpu(i=0): #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus(): #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## Tensors and GPUs

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

### Storage on the GPU

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [11]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.7491, 0.4584, 0.2015],
        [0.0448, 0.6921, 0.3528]], device='cuda:0')

### Copying

In [9]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [12]:
Y + Z

tensor([[1.7491, 1.4584, 1.2015],
        [1.0448, 1.6921, 1.3528]], device='cuda:0')

## Neural Networks and GPUs

In [13]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

In [14]:
net(X)

tensor([[-0.5553],
        [-0.5553]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [15]:
net[0].weight.data.device

device(type='cuda', index=0)

In [16]:
@d2l.add_to_class(d2l.Trainer) #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer) #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer) #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model